## Imort the required Libraries as shown below

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.90 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00  35.97 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.28 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.84 MB/s
vincent-0.4.4- 100% |###################

### Get the latitude and Longitude of Mumbai India using geopy Library

In [2]:
address = 'Mumbai'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


### Set the parameters to access the four Square API and set Search query parameter as "Italian"

In [3]:
CLIENT_ID = 'GMPYRYWGWETG24X1FKGEMRAQCOID3GGS35DHM4PW0SUI3RXZ' # your Foursquare ID
CLIENT_SECRET = 'KQUCK1HAZ4PX1DZDQC0UCCX12T4SHJFAYP24GK54OAAS5VGP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
search_query = 'Italian'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GMPYRYWGWETG24X1FKGEMRAQCOID3GGS35DHM4PW0SUI3RXZ
CLIENT_SECRET:KQUCK1HAZ4PX1DZDQC0UCCX12T4SHJFAYP24GK54OAAS5VGP


### Create the URl to get the response from FourSquare API

In [4]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&query={}&ll={},{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,search_query,
    latitude, 
    longitude)
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=GMPYRYWGWETG24X1FKGEMRAQCOID3GGS35DHM4PW0SUI3RXZ&client_secret=KQUCK1HAZ4PX1DZDQC0UCCX12T4SHJFAYP24GK54OAAS5VGP&v=20180605&query=Italian&ll=18.9387711,72.8353355'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0f41854434b903c2a5e1c0'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1d0941735',
      'name': 'Dessert Shop',
      'pluralName': 'Dessert Shops',
      'primary': True,
      'shortName': 'Desserts'}],
    'hasPerk': False,
    'id': '4e0d50acae603a50b5381414',
    'location': {'address': 'sterling complex',
     'cc': 'IN',
     'country': 'India',
     'distance': 248,
     'formattedAddress': ['sterling complex', 'India'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 18.93803255629217,
       'lng': 72.83311082866292}],
     'lat': 18.93803255629217,
     'lng': 72.83311082866292},
    'name': 'Gelato Italiano',
    'referralId': 'v-1544503685'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d124941

### Get the relevant part of Response and normalise it

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.shape

(30, 18)

### keep only columns that include venue name, and anything that is associated with location

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gelato Italiano,Dessert Shop,sterling complex,IN,NaN,India,NaN,248,"[sterling complex, India]","[{'lng': 72.83311082866292, 'label': 'display'...",18.938033,72.833111,NaN,NaN,4e0d50acae603a50b5381414
1,The Indo-Italian Chamber of Commerce and Industry,Office,502 Veer Savarkar Marg,IN,Mumbai,India,NaN,8429,"[502 Veer Savarkar Marg, Mumbai 400 025, Mahār...","[{'lng': 72.82638, 'label': 'display', 'lat': ...",19.014021,72.826380,400 025,Mahārāshtra,4b0587e6f964a52056a622e3
2,Italian Pizzas,Italian Restaurant,"Sion Bandra Link Rd., Opp. Hallmark Plaza",IN,Mumbai,India,Bandra East,13456,"[Sion Bandra Link Rd., Opp. Hallmark Plaza (Ba...","[{'lng': 72.8513460275445, 'label': 'display',...",19.058700,72.851346,NaN,Mahārāshtra,4e4f52c0d22daf51d274cf7b
3,Italian Pizzas,Pizza Place,"Ashapura CHS, Indira Nagar,",IN,Bāndra,India,"Near Gurunanak Hospital,",13654,"[Ashapura CHS, Indira Nagar, (Near Gurunanak H...","[{'lng': 72.85223159742127, 'label': 'display'...",19.060388,72.852232,NaN,Mahārāshtra,513ca918e4b0242783e8c66c
4,Gelato Italiano,Ice Cream Shop,"K Dubash Marg, Kala Ghoda, Fort",IN,Mumbai,India,NaN,1289,"[K Dubash Marg, Kala Ghoda, Fort, Mumbai, Mahā...","[{'lng': 72.83290311140377, 'label': 'display'...",18.927414,72.832903,NaN,Mahārāshtra,4bdc584ac79cc928501b86e9
5,Italian Bakery,Bakery,Juhu Koliwada,IN,NaN,India,NaN,16375,"[Juhu Koliwada, India]","[{'lng': 72.82871987895068, 'label': 'display'...",19.085743,72.828720,NaN,NaN,50961017e4b0f7170362fd0e
6,Gelato Italiano,None,"Inox, Nariman Point",IN,Mumbai,India,NaN,1867,"[Inox, Nariman Point, Mumbai, Mahārāshtra, India]","[{'lng': 72.82275937012078, 'label': 'display'...",18.926937,72.822759,NaN,Mahārāshtra,4dc57812ae608779d13b8de0
7,Gelato Italiano,None,NaN,IN,NaN,India,NaN,2469,"[Mahārāshtra, India]","[{'lng': 72.81690258143738, 'label': 'display'...",18.952483,72.816903,NaN,Mahārāshtra,4d89cfd04757721ecec0a869
8,Amore Italian Gelato,Ice Cream Shop,NaN,IN,NaN,India,NaN,27297,[India],"[{'lng': 72.83577651778005, 'label': 'display'...",19.183993,72.835777,NaN,NaN,4ea30e258231f6dc6a4bde4b
9,Fratelli Fresh Italian Restaurant,Italian Restaurant,"#2 & 3B, Near Chinmayanand Ashram, Powai",IN,Mumbai,India,Renaissance Mumbai Convention Centre Hotel,22923,"[#2 & 3B, Near Chinmayanand Ashram, Powai (Ren...","[{'lng': 72.90195149156897, 'label': 'display'...",19.134830,72.901951,400087,Mahārāshtra,4cebd3aabaa6a1cd2a65406c


In [8]:
dataframe_filtered.name

0                                       Gelato Italiano
1     The Indo-Italian Chamber of Commerce and Industry
2                                        Italian Pizzas
3                                        Italian Pizzas
4                                       Gelato Italiano
5                                        Italian Bakery
6                                       Gelato Italiano
7                                       Gelato Italiano
8                                  Amore Italian Gelato
9                     Fratelli Fresh Italian Restaurant
10                                      Gelato Italiano
11                                      Gelato Italiano
12                           Sabarro Italian food place
13                                     Italian Compound
14                                     Italian Terrazzo
15                                      Gelato Italiano
16                                      Gelato italiano
17    The Indo-Italian Chamber of Commerce and I

### Display all the Restaurants on Map

In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Get the Rating of all the Italian Restaurants whose details are available and store them in a data frame.

In [17]:
df_rating=[]
for index, row in dataframe_filtered.iterrows():
    venue_id = (row['id'])
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #print(result['response']['venue'].keys())
    try:
        df_rating.append({'id':result['response']['venue']['id'] , 'rating':result['response']['venue']['rating']})
    except:
        print('This venue has not been rated yet.')
pd.DataFrame(df_rating)

This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.


,id,rating
0,4cebd3aabaa6a1cd2a65406c,8.3
1,4ff97771e4b0a7f7418e8446,8.3
2,53e897c4498e85fdd68f8fe4,7.6
3,4d415afa3fb4ba7a192e8a25,7.7
4,4dccd3981f6eb1227052e734,7.7
5,4ce81d40f8653704415baec4,6.6
6,4d6d37d4125eb60c6623324e,6.2
7,4be43678bcef2d7f253502e5,5.4
8,5356af4b11d2df16e9b8e151,6.6


### Once the dataframe is created, sort the dataframe based on Rating in ascending order and get the top 3 least rated Italian Restaurants

In [18]:
df = pd.DataFrame(df_rating)
df_final=pd.merge(dataframe_filtered, df, on='id')
df_final_sort = df_final.sort_values('rating',ascending=True)
df_final_sort = df_final_sort.head(3)
df_final_sort = df_final_sort.reset_index(drop=True)
df_final_sort

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,rating
0,Mezbaan,Indian Restaurant,carter road,IN,Mumbai,India,naushad Ali marg,14744,"[carter road (naushad Ali marg), Mumbai 400052...","[{'lng': 72.82250299100305, 'label': 'display'...",19.070665,72.822503,400052,Mahārāshtra,4be43678bcef2d7f253502e5,5.4
1,Gelato Italiano,Dessert Shop,NaN,IN,Mumbai,India,NaN,12166,"[Mumbai, Mahārāshtra, India]","[{'lng': 72.86411064110362, 'label': 'display'...",19.044628,72.864111,NaN,Mahārāshtra,4d6d37d4125eb60c6623324e,6.2
2,Gelato Italiano,Dessert Shop,NaN,IN,Mumbai,India,NaN,9912,"[Mumbai, Mahārāshtra, India]","[{'lng': 72.85396820107265, 'label': 'display'...",19.026059,72.853968,NaN,Mahārāshtra,4ce81d40f8653704415baec4,6.6


### Mark the least rated Italian Restaurant in Map, indicating the location where we can open a new restaurant.   Since these locations are exposed for Italian restaurants but miss good Itaian restaurants, there is a high chance that food chain can open a restaurant here and will run into profits
    

In [12]:
venues_map_restaurants = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_final_sort.lat, df_final_sort.lng, df_final_sort.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_restaurants)

# display map
venues_map_restaurants